In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('../EDA/numData.csv')
Y = pd.read_csv('../EDA/y.csv')

In [3]:
dataset = dataset.merge(Y, how='left', on=["country","year"])

In [4]:
X_AfterColdWar = dataset[dataset["year"] >= 1990]
X_BeforeColdWar = dataset[dataset["year"] < 1990]

In [5]:
X_without = dataset.drop(columns=["year","country"], axis=0)
X_AfterColdWar_without = X_AfterColdWar.drop(columns=["year","country"], axis=0)
X_BeforeColdWar_without = X_BeforeColdWar.drop(columns=["year","country"], axis=0)

In [6]:
bigMissing = X_without.isna().sum()[(X_without.isna().sum() / X_without.shape[0]) > 0.8].keys()
X_without = X_without.drop(columns=bigMissing, axis=0)

In [7]:
X_without = X_without.replace(np.inf, 0)
X_without = X_without.replace(-np.inf, 0)
X_without = X_without.fillna(0)

X_BeforeColdWar_without = X_BeforeColdWar_without.replace(np.inf, 0)
X_BeforeColdWar_without = X_BeforeColdWar_without.replace(-np.inf, 0)
X_BeforeColdWar_without = X_BeforeColdWar_without.fillna(0)

X_AfterColdWar_without = X_AfterColdWar_without.replace(np.inf, 0)
X_AfterColdWar_without = X_AfterColdWar_without.replace(-np.inf, 0)
X_AfterColdWar_without = X_AfterColdWar_without.fillna(0)

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = StandardScaler().fit_transform(X_without)
X_aCW = StandardScaler().fit_transform(X_AfterColdWar_without)
X_bCW = StandardScaler().fit_transform(X_BeforeColdWar_without)

pca = PCA()
principalComponents = pca.fit_transform(X)

principalComponents_bCW = pca.fit_transform(X_bCW)
principalComponents_aCW = pca.fit_transform(X_aCW)

PCA_components = pd.DataFrame(principalComponents)
PCA_components_bCW = pd.DataFrame(principalComponents_bCW)
PCA_components_aCW = pd.DataFrame(principalComponents_aCW)

In [9]:
# from sklearn import mixture

# components = 9
# features = 5

# gmm = mixture.GaussianMixture(n_components=components, covariance_type='full')
# labels = gmm.fit_predict(PCA_components_bCW.iloc[:,:features])

# X_BeforeColdWar_without["gmmLabels-"+str(components)+str(features)] = labels
# X_AfterColdWar["gmmLabels-"+str(components)+str(features)] = labels

In [10]:
from sklearn import mixture

components = 9
features = 5

gmm = mixture.GaussianMixture(n_components=components, covariance_type='full')
labels = gmm.fit_predict(PCA_components.iloc[:,:features])

X_without["gmmLabels-"+str(components)+str(features)] = labels
dataset["gmmLabels-"+str(components)+str(features)] = labels


In [11]:
from sklearn.cluster import KMeans

components = 9
features = 5

gmm = KMeans(n_clusters=components)
labelsKmm = gmm.fit_predict(PCA_components.iloc[:,:features])

X_without["kmmLabels-"+str(components)+str(features)] = labelsKmm
dataset["kmmLabels-"+str(components)+str(features)] = labelsKmm

In [12]:
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
import scipy.cluster.hierarchy as shc

components = 6
features = 5

cluster = shc.linkage(PCA_components.iloc[:,:features], method='complete')
cluster_array = fcluster(cluster, 15, criterion='maxclust')

X_without["LinkLabels-"+str(components)+str(features)] = cluster_array
dataset["LinkLabels-"+str(components)+str(features)] = cluster_array


In [ ]:
dataset.to_csv("cheat.csv")

In [ ]:
checkingMatrix = dataset.iloc[:,-12:]

In [ ]:
checkingMatrix

In [ ]:
import sklearn.metrics as skMetrics

labels = checkingMatrix.iloc[:,-3:]

zerosArray = np.zeros((3,3))


for x in range(3):
    for y in range(3):
        randIndex = skMetrics.adjusted_rand_score(labels.iloc[:,x],labels.iloc[:,y])
        zerosArray[x][y] = randIndex

np.savetxt("rand.csv", zerosArray, delimiter=",", fmt='%s')

In [ ]:
import scipy.stats as scStat

setToCompare = "gmmLabels-95"
clusters = len(checkingMatrix[setToCompare].unique())
democracyScore = "v2x_libdem"

clusters = len(checkingMatrix[setToCompare].unique())

bars = [np.mean(checkingMatrix[checkingMatrix[setToCompare]==i][democracyScore].dropna().values) for i in range(clusters)]
plt.bar(range(clusters),bars)
plt.xticks(range(clusters))
plt.show()


# zeroArray = np.zeros((clusters+1,clusters+1))
zeroArray = [[0]*(clusters+1)]*(clusters+1)

barlett = [checkingMatrix[checkingMatrix[setToCompare]==i][democracyScore].dropna().values for i in range(clusters)]
barlettTest = scStat.levene(*barlett)

print(barlettTest)

for i in range(clusters):
    # print(len(checkingMatrix[checkingMatrix[setToCompare]==i]["v2x_partipdem"]))
    compareSet = checkingMatrix[checkingMatrix[setToCompare]==i][democracyScore].dropna().values
    if len(compareSet) > 8:
        normality_v2X_PartiDem = scStat.normaltest(compareSet)
        print(normality_v2X_PartiDem, barlettTest)
        for z in range(clusters):
            tTest = 0
            if i != z:
                compareSet2 = checkingMatrix[checkingMatrix[setToCompare]==z][democracyScore].dropna().values
                levene = scStat.levene(compareSet,compareSet2)
                print(levene)

                if len(compareSet2) > 8:
                    tTest = scStat.ttest_ind(compareSet,compareSet2)
                    zeroArray[i][z] = str(tTest[1])+ ";" + str(levene[1])
                    print("Compare "+ democracyScore +", of cluster", i, " against:", z, ":", tTest)
                    print("")
            else:
                zeroArray[i][z] = 1
    print("")

save = np.array(zeroArray)
np.savetxt("foo.csv", save, delimiter=",", fmt='%s')